In [3]:
from elasticsearch import Elasticsearch
from faker import Faker

# Établir la connexion à Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

TypeError: NodeConfig.__init__() missing 1 required positional argument: 'scheme'

In [1]:
import pandas as pd

df=pd.read_csv('emotion.csv')


df1=df.iloc[:8000]
df2=df.iloc[8000:16000]
df3=df.iloc[16000:]

In [43]:
df.shape

(21459, 2)

In [2]:
from elasticsearch import Elasticsearch
from faker import Faker
import pickle
# Établir la connexion à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Nom de l'index
index_name = 'notes'

# Créer une instance de Fakercurl -X POST 'http://localhost:9200/articles/_doc?pretty' -H 'Content-Type: application/json' -d '
{
    "created_at": "2021-06-07T16:24:32.000Z",
    "title": "mon article sur Elasticsearch",
    "content": "ceci est le contenu de mon article sur Elasticsearch",
    "category": "elasticsearch",
    "author": "Hatim"
  }
fake = Faker()

with open('pic_nlp.pkl', 'rb') as file:
    modele_ml = pickle.load(file)
# Nombre de documents à générer
liste_nom=[]
liste_prenom=[]

count = 0
rang= 0
for i in range (500):
    nom = fake.last_name()
    prenom = fake.first_name()
    liste_nom.append(nom)
    liste_prenom.append(prenom)
    
for index, row in df1.iterrows():
    print(index)
    if index %50==0:
        rang+=1
    document = {
        'patient_lastname': liste_nom[rang],
        'patient_firstname': liste_prenom[rang],
        'text': row['Text'],
        'date': fake.date_object(),
        'patient_left': fake.boolean(),
        'emotion': row['Emotion'],
        'confidence': fake.pyfloat()
    }
    
    # Indexer le document dans Elasticsearch
    response = es.index(index=index_name, body=document)
    
    if response['result'] == 'created':
        print('Document indexé avec succès.')
    else:
        print("Erreur lors de l'indexation du document.")
   
    count += 1

es.indices.refresh(index=index_name)
es.transport.close()

print(f"Indexation terminée pour documents.")


# commande pour voir ts les index

KeyboardInterrupt: 

In [ ]:

GET /'notes'/_search
{
  "query": {
    "match": {
      "notes": "important"
    }
  }
}

In [57]:


from elasticsearch import Elasticsearch
import pandas as pd

# Se connecter à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Nom de l'index à récupérer les données
index_name = 'notes'

# Récupérer les informations de mappage de l'index
mapping = es.indices.get_mapping(index=index_name)
properties = mapping[index_name]['mappings']['properties']

# Extraire les noms des champs du mappage
champs = list(properties.keys())

# Définir la requête de recherche pour récupérer toutes les lignes de l'index
requete = {
    "query": {
        "match_all": {}
    },
    "size": 10000,  # Nombre de résultats à récupérer
    "from":10000
}

# Exécuter la requête de recherche
result = es.search(index=index_name, body=requete)

# Extraire les données des documents
documents = [hit['_source'] for hit in result['hits']['hits']]

# Créer un DataFrame à partir des données
df1 = pd.DataFrame(documents, columns=champs)

# Afficher le DataFrame
print(df1)


/tmp/ipykernel_36568/2443269931.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  mapping = es.indices.get_mapping(index=index_name)
/tmp/ipykernel_36568/2443269931.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=requete)


BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'Result window is too large, from + size must be less than or equal to: [10000] but was [20000]. See the scroll api for a more efficient way to request large data sets. This limit can be set by changing the [index.max_result_window] index level setting.')

In [59]:
from elasticsearch import Elasticsearch
import pandas as pd

# Se connecter à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Nom de l'index à récupérer les données
index_name = 'notes'

# Récupérer les informations de mappage de l'index
mapping = es.indices.get_mapping(index=index_name)
properties = mapping[index_name]['mappings']['properties']

# Extraire les noms des champs du mappage
champs = list(properties.keys())

# Définir la requête de recherche initiale avec scroll
requete = {
    "query": {
        "match_all": {}
    },
    "size": 1000,  # Taille de chaque lot
    "scroll": "1m"  # Durée de validité du scroll
}

# Exécuter la requête de recherche initiale
result = es.search(index=index_name, body=requete, scroll="1m")

# Extraire les données des documents
documents = [hit['_source'] for hit in result['hits']['hits']]

# Tant qu'il y a plus de résultats, continuer à récupérer les lots suivants
scroll_id = result['_scroll_id']
while True:
    # Récupérer le lot suivant avec scroll
    result = es.scroll(scroll_id=scroll_id, scroll='1m')
    scroll_id = result['_scroll_id']
    
    # Extraire les données des documents du lot suivant
    documents.extend([hit['_source'] for hit in result['hits']['hits']])
    
    # Vérifier si tous les résultats ont été récupérés
    if len(result['hits']['hits']) == 0:
        break

# Créer un DataFrame à partir des données
df = pd.DataFrame(documents, columns=champs)

# Afficher le DataFrame
print(df)


/tmp/ipykernel_36568/3223417957.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  mapping = es.indices.get_mapping(index=index_name)
/tmp/ipykernel_36568/3223417957.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=requete, scroll="1m")


ValueError: Received multiple values for 'scroll', specify parameters directly instead of using 'body' or 'params'

In [51]:
from elasticsearch import Elasticsearch

# Se connecter à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Index à interroger
index_name = 'notes'

# Requête d'agrégation
requete = {
    "size": 10,
    "aggs": {
        "patients_uniques": {
            "terms": {
                "script": {
                    "source": "doc['lastname.keyword'].value + ' ' + doc['surname.keyword'].value",
                    "lang": "painless"
                },
                "size": 10000
            },
            "aggs": {
                "emotions_uniques": {
                    "significant_terms": {
                        "field": "emotion",
                        "percentage": {},
                        "size": 10000
                    }
                }
            }
        }
    }
}

# Exécuter la requête
result = es.search(index=index_name, body=requete)

# Récupérer les résultats de l'agrégation
aggregation = result['aggregations']['patients_uniques']['buckets']

# Parcourir les résultats
for bucket in aggregation:
    patient = bucket['key']
    emotions = bucket['emotions_uniques']['buckets']
    total_occurrences = sum([bucket['doc_count'] for bucket in emotions])
    
    print(f"Patient: {patient}")
    for emotion_bucket in emotions:
        emotion = emotion_bucket['key']
        occurrence_percentage = (emotion_bucket['doc_count'] / total_occurrences) * 100
        print(f"Emotion: {emotion}, Pourcentage: {occurrence_percentage:.2f}%")
    print()



/tmp/ipykernel_36568/2688201981.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=requete)


BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')

In [20]:
from elasticsearch import Elasticsearch
import pickle
import numpy as np

# Établir la connexion à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Nom de l'index
index_name = 'notes'

# Charger le modèle ML à partir du fichier pickle
with open('pic_nlp.pkl', 'rb') as file:
    modele_ml = pickle.load(file)

# Récupérer tous les documents de l'index "notes"
response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=9000)

# Parcourir les résultats et mettre à jour le champ "confidence"
for hit in response['hits']['hits']:
    print(hit)
    document_id = hit['_id']
    document_text = hit['_source']['text']
    doc_emotion=hit['_source']['emotion']
    conf=hit['_source']['confidence']
    print(conf)
    # Appliquer le modèle ML pour obtenir le résultat
    emotion = modele_ml.predict(doc_emotion)
    decision_scores = modele_ml.decision_function([document_text])[0]

# Transformer les scores de décision en probabilités normalisées
    probabilities = decision_scores - decision_scores.min()
    probabilities /= probabilities.max()

# Obtenir le pourcentage de confiance en la prédiction
    confidence_rslt = probabilities[np.argmax(probabilities)] * 100
    
    # Mettre à jour le champ "confidence" du document
    update_body = {"doc": {"emotion": emotion, "confidence":confidence_rslt}}
    es.update(index=index_name, id=document_id, body=update_body)

print("Mise à jour des champs 'confidence' terminée.")


Mise à jour des champs 'confidence' terminée.


/tmp/ipykernel_36568/3062926506.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=9000)
/tmp/ipykernel_36568/3062926506.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=9000)


# tout supprimer

In [4]:
es = Elasticsearch(hosts=["http://localhost:9200"])

In [5]:
# #  # # Connexion à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# # # # Suppression de l'index "notes"
index_name = "notes"
response = es.indices.delete(index=index_name, ignore=[400, 404])

# # # # Vérification de la suppression
if response["acknowledged"]:
    print(f"L'index '{index_name}' a été supprimé avec succès.")
else:
    print(f"La suppression de l'index '{index_name}' a échoué.")

# # # # Fermeture de la connexion à Elasticsearch
es.transport.close()

/tmp/ipykernel_5890/1498190664.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.delete(index=index_name, ignore=[400, 404])


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f2fe1dc4d30>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
es = Elasticsearch(hosts=["http://localhost:9200"])es = Elasticsearch(hosts=["http://localhost:9200"])